In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd

import gc
import timeit
import sys
import os

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
from scipy.optimize import minimize

# Residual Sum of Squares calculations: synthetic data against data for seen deer per hunting hour

In [4]:
# Revoking stored reported data
# https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html

%store -r data_Averoy
%store -r data_Tingvoll
%store -r data_Surnadal
%store -r data_Sunndal
%store -r data_Vestnes
%store -r data_Laerdal

In [5]:
def sum_squares_seen_deer(
    number_of_years,
    municipality_file,
    empirical_observations,
    compare,
):
    """
    Calculating the minimum residual_sum_square between synthetic data and
    reported data.
    """
    # Revoking the targeted municipality data frame created by the synthetic population generator
    save_dir = "synthetic_data/"
    data_frame = pd.read_pickle(save_dir + municipality_file)
    # Discarding all rows where the stags_per_hind_threshold > 0
    data_frame = data_frame.loc[data_frame["sh_threshold"] < 0.01]

    # Defining column to extract values from
    if compare == "before_hunting":
        comp_choice = "tot_pop_bh"
    if compare == "after_hunting":
        comp_choice = "tot_pop_ah"

    RSS_list = []
    for i in range(0, int(len(data_frame) / number_of_years)):
        # Catch the predicted population figures for all observation years
        seen_deer_p = data_frame.iloc[i * number_of_years : (i + 1) * number_of_years][
            comp_choice
        ].values

        seen_deer_e = np.array(empirical_observations)

        # Find the scaling factor that minimises RSS and do the scaling
        scaling_factor = np.sum(np.multiply(seen_deer_p, seen_deer_e)) / np.sum(
            np.multiply(seen_deer_p, seen_deer_p)
        )
        seen_deer_p_scaled = scaling_factor * seen_deer_p

        # Find minimum Residual Sum Square value
        RSS_min = np.sum((seen_deer_p_scaled - seen_deer_e) ** 2)

        RSS_list.append([scaling_factor, RSS_min])

    return RSS_list

In [6]:
def make_RSS_frame(municipality_file, RSS_list):

    # Revoking the targeted municipality data frame created by the synthetic population generator
    save_dir = "synthetic_data/"
    data_frame = pd.read_pickle(save_dir + municipality_file)
    data_frame = data_frame.loc[data_frame["sh_threshold"] < 0.01]

    # Remove all rows except those where obs_year == 2021
    df_filtered = data_frame[data_frame["obs_year"] == 2021]

    # Remove columns not used
    cols = [
        "f_calves",
        "y_hinds",
        "a_hinds",
        "m_calves",
        "y_stags",
        "a_stags",
        "ws_fc",
        "ws_yh",
        "ws_ah",
        "ws_mc",
        "ws_ys",
        "ws_as",
        "c_yh",
        "c_ah",
    ]
    df_filtered2 = df_filtered.drop(cols, axis=1)

    # Resert index, otherwise pd.concat does not work
    df_filtered2 = df_filtered2.reset_index(drop=True)

    # Add two columns from RSS_list
    df_RSS = pd.concat(
        [df_filtered2, pd.DataFrame(RSS_list, columns=["scaling", "RSS"])], axis=1
    )

    # Sort the frame on the RSS value
    sorted_sum_squares_frame = df_RSS.sort_values(by=["RSS"]).reset_index(drop=True)

    return sorted_sum_squares_frame

In [7]:
def extract_top_hits(
    i,
    sorted_sum_squares_frame,
    filtering_strategy,
    frac_init_pop,
    number_of_top_hits,
):
    """
    Filtering the sum_squares frame based on an assumption about the
    size of the Dec 31 2020 population vs the Dec 31 2006 population,
    and delivering only the number_of_top_hits best fits.
    """

    if filtering_strategy == "uninformed":
        sorted_sum_squares_frame_filtered = sorted_sum_squares_frame

    if filtering_strategy == "informed":
        # Educated guess filtering - can play with these criteria
        sorted_sum_squares_frame_filtered = sorted_sum_squares_frame[
            (
                sorted_sum_squares_frame.tot_pop_ah
                > frac_init_pop[i] * sorted_sum_squares_frame.init_pop
            )
        ].reset_index(drop=True)

    return sorted_sum_squares_frame_filtered[0:number_of_top_hits]

In [8]:
def run_the_show():
    # Using all 6 municipalities
    municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]

    data_municipality = [
        data_Averoy,
        data_Tingvoll,
        data_Surnadal,
        data_Sunndal,
        data_Vestnes,
        data_Laerdal,
    ]

    municipality_frame = [
        "df_original_sorted_Averoy.pkl",
        "df_original_sorted_Tingvoll.pkl",
        "df_original_sorted_Surnadal.pkl",
        "df_original_sorted_Sunndal.pkl",
        "df_original_sorted_Vestnes.pkl",
        "df_original_sorted_Laerdal.pkl",
    ]

    compare = "before_hunting"
    filtering_strategy = "informed"
    frac_init_pop = [0.5] * len(municipalities)  # minumum tot_pop 2020/2006 ratio
    number_of_top_hits = 20

    for q in range(len(municipalities)):
        [
            municipality,
            first_year,
            last_year,
            number_of_years,
            years,
            seen_deer_obs,
            seen_deer_obs_outfield,
            seen_deer_obs_infield,
            hinds_per_stag_obs,
            total_harvest,
            fraction_female_calves_harvested,
            fraction_young_hinds_harvested,
            fraction_adult_hinds_harvested,
            fraction_male_calves_harvested,
            fraction_young_stags_harvested,
            fraction_adult_stags_harvested,
            spring_counts,
        ] = data_municipality[q]

        if municipality == "Surnadal":
            RSS_list = sum_squares_seen_deer(
                number_of_years,
                municipality_frame[q],
                seen_deer_obs_infield,
                compare,
            )
        else:
            RSS_list = sum_squares_seen_deer(
                number_of_years,
                municipality_frame[q],
                seen_deer_obs,
                compare,
            )

        sorted_sum_squares_frame = make_RSS_frame(municipality_frame[q], RSS_list)

        top_hits_frame_filtered = extract_top_hits(
            q,
            sorted_sum_squares_frame,
            filtering_strategy,
            frac_init_pop,
            number_of_top_hits,
        )

        # Storing top hits frames
        save_dir = "synthetic_data/"
        top_hits_frame_filtered.to_pickle(
            save_dir + "top_hits_" + municipality + "_sd_no_emigration" + ".pkl"
        )

        print("Municipality = ", municipality)
        display(top_hits_frame_filtered)
        print()
        print(
            "Percentage difference between min and max RSS value: ",
            round(
                (
                    (
                        top_hits_frame_filtered["RSS"].max()
                        - top_hits_frame_filtered["RSS"].min()
                    )
                    / top_hits_frame_filtered["RSS"].min()
                )
                * 100,
                2,
            ),
        )
        print()
        print("Mean column values of top_hits_frame:")
        display(top_hits_frame_filtered.mean())
        print()
        print()

In [9]:
%%time
# Running all scripts by calling up run_the_show
# This is done to get rid of memory leaks
run_the_show()

Municipality =  Averoy


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,scaling,RSS
0,1250,1.3,0.0,143,48,495,143,200,217,2021,1415,1783,1350,0.6039,0.3748,0.0,0,0.000654,0.967004
1,1400,1.3,0.0,231,132,397,231,122,285,2021,1194,1501,1068,0.6065,0.3306,0.0,0,0.000688,0.976974
2,1250,1.5,0.0,187,83,441,187,70,280,2021,1242,1621,1188,0.7657,0.4896,0.0,0,0.000696,0.978248
3,1400,1.4,0.0,203,52,527,203,99,314,2021,1235,1551,1120,0.5537,0.2954,0.0,0,0.000660,0.980555
4,1300,1.5,0.0,162,152,432,162,116,273,2021,1034,1304,879,0.5730,0.2669,0.0,0,0.000740,0.982288
5,1400,1.4,0.0,245,90,440,245,113,265,2021,1190,1470,1040,0.4981,0.2348,0.0,0,0.000677,0.982796
6,1300,1.6,0.0,136,107,524,136,157,237,2021,1074,1358,933,0.5859,0.2909,0.0,0,0.000740,0.984973
7,1250,1.4,0.0,125,58,525,125,91,325,2021,1290,1584,1151,0.4999,0.2553,0.0,0,0.000683,0.988435
8,1300,1.3,0.0,149,107,458,149,191,243,2021,1248,1579,1146,0.6117,0.3484,0.0,0,0.000690,0.988861
9,1350,1.3,0.0,202,133,400,202,172,238,2021,1139,1440,1008,0.5881,0.3007,0.0,0,0.000716,0.992644



Percentage difference between min and max RSS value:  5.24

Mean column values of top_hits_frame:


init_pop        1320.000000
init_hps           1.420000
sh_threshold       0.000000
i_fcalves        180.500000
i_yhinds         101.600000
i_ahinds         457.800000
i_mcalves        180.500000
i_ystags         131.300000
i_astags         265.700000
obs_year        2021.000000
spring_pop      1206.450000
tot_pop_bh      1523.700000
tot_pop_ah      1093.750000
hps_bh             0.602430
hps_ah             0.330990
mig%               0.000000
num_migs           0.000000
scaling            0.000696
RSS                0.994117
dtype: float64



Municipality =  Tingvoll


...



Municipality =  Laerdal


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,scaling,RSS
0,1600,1.8,0.0,192,93,687,192,217,217,2021,1360,1832,1300,1.0368,0.9388,0.0,0,0.000116,0.019292
1,1650,1.4,0.0,222,133,569,222,230,271,2021,1518,2036,1504,0.9663,0.8691,0.0,0,0.000112,0.019308
2,1650,1.5,0.0,189,198,564,189,203,304,2021,1477,1952,1420,0.8813,0.7680,0.0,0,0.000112,0.019460
3,1550,1.9,0.0,170,198,594,170,125,291,2021,1466,1955,1423,0.9637,0.8619,0.0,0,0.000112,0.019496
4,1600,1.3,0.0,160,86,636,160,122,434,2021,1380,1857,1325,0.9248,0.8084,0.0,0,0.000116,0.019498
5,1550,1.8,0.0,263,138,519,263,182,182,2021,1319,1809,1277,1.1023,1.0147,0.0,0,0.000121,0.019508
6,1700,1.6,0.0,238,203,549,238,122,348,2021,1386,1837,1305,0.8649,0.7400,0.0,0,0.000114,0.019552
7,1650,1.4,0.0,181,105,645,181,225,311,2021,1494,1996,1464,0.8974,0.7878,0.0,0,0.000109,0.019586
8,1700,1.5,0.0,178,217,588,178,139,397,2021,1542,2056,1524,0.9333,0.8329,0.0,0,0.000108,0.019587
9,1650,2.0,0.0,231,182,609,231,71,324,2021,1506,1965,1433,0.8044,0.6838,0.0,0,0.000109,0.019608



Percentage difference between min and max RSS value:  2.34

Mean column values of top_hits_frame:


init_pop        1620.000000
init_hps           1.595000
sh_threshold       0.000000
i_fcalves        194.700000
i_yhinds         149.300000
i_ahinds         600.950000
i_mcalves        194.700000
i_ystags         168.050000
i_astags         309.900000
obs_year        2021.000000
spring_pop      1454.200000
tot_pop_bh      1951.850000
tot_pop_ah      1419.850000
hps_bh             0.975455
hps_ah             0.875155
mig%               0.000000
num_migs           0.000000
scaling            0.000113
RSS                0.019592
dtype: float64



CPU times: user 35min 1s, sys: 9min 42s, total: 44min 44s
Wall time: 44min 39s
